In [39]:
import pyspark;
sc = pyspark.SparkContext.getOrCreate();
from pyspark.sql import SparkSession;
spark = SparkSession(sc)

In [4]:
retail_df = (spark \
   .read \
   .option("header", "true") \
   .option("inferSchema", "true") \
   .csv("/home/nolfonzo/src/pyspark/Essential-PySpark-for-Scalable-Data-Analytics/data/online_retail/online_retail.csv") \
)

In [6]:
retail_df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|01/12/10 08:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01/12/10 08:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01/12/10 08:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01/12/10 08:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01/12/10 08:26|     4.

In [11]:
(retail_df 
    .write
    .mode("overwrite")
    .parquet("/tmp/data-lake/online_retail.parquet")
)

22/05/20 13:20:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [30]:
csv_df = (spark \
    .read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/home/nolfonzo/src/pyspark/Essential-PySpark-for-Scalable-Data-Analytics/data/online_retail/online_retail_II.csv") \
)
retail_df = csv_df.selectExpr("Invoice as InvoiceNo", "StockCode", "Description", "Quantity", "InvoiceDate", "Price as UnitPrice", "`Customer ID` as CustomerID", "Country")
retail_df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   489434|    85048|15CM CHRISTMAS GL...|      12|01/12/09 07:45|     6.95|     13085|United Kingdom|
|   489434|   79323P|  PINK CHERRY LIGHTS|      12|01/12/09 07:45|     6.75|     13085|United Kingdom|
|   489434|   79323W| WHITE CHERRY LIGHTS|      12|01/12/09 07:45|     6.75|     13085|United Kingdom|
|   489434|    22041|"RECORD FRAME 7""...|      48|01/12/09 07:45|      2.1|     13085|United Kingdom|
|   489434|    21232|STRAWBERRY CERAMI...|      24|01/12/09 07:45|     1.25|     13085|United Kingdom|
|   489434|    22064|PINK DOUGHNUT TRI...|      24|01/12/09 07:45|     1.65|     13085|United Kingdom|
|   489434|    21871| SAVE THE PLANET MUG|      24|01/12/09 07:45|     1.

In [31]:
from pyspark.sql.functions import to_json, struct, from_json, monotonically_increasing_id
from pyspark.sql.types import StructType, StructField, StringType
kafka_df = retail_df.withColumn("key", monotonically_increasing_id().cast("STRING")).withColumn("value", to_json(struct([retail_df[x] for x in retail_df.columns])).cast("STRING"))
jsonSchema = StructType([ StructField("eventName", StringType(), True), \
                          StructField("eventParams", StringType(), True) \
                        ])
kafka_df.select("key", "value").show()

+---+--------------------+
|key|               value|
+---+--------------------+
|  0|{"InvoiceNo":"489...|
|  1|{"InvoiceNo":"489...|
|  2|{"InvoiceNo":"489...|
|  3|{"InvoiceNo":"489...|
|  4|{"InvoiceNo":"489...|
|  5|{"InvoiceNo":"489...|
|  6|{"InvoiceNo":"489...|
|  7|{"InvoiceNo":"489...|
|  8|{"InvoiceNo":"489...|
|  9|{"InvoiceNo":"489...|
| 10|{"InvoiceNo":"489...|
| 11|{"InvoiceNo":"489...|
| 12|{"InvoiceNo":"489...|
| 13|{"InvoiceNo":"489...|
| 14|{"InvoiceNo":"489...|
| 15|{"InvoiceNo":"489...|
| 16|{"InvoiceNo":"489...|
| 17|{"InvoiceNo":"489...|
| 18|{"InvoiceNo":"489...|
| 19|{"InvoiceNo":"489...|
+---+--------------------+
only showing top 20 rows



In [38]:
kafka_df.select("key", "value")\
  .write\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("topic", "retail_events")\
  .save()


AnalysisException:  Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".        